In [ ]:
from telethon import TelegramClient, events, sync
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import PeerChannel
import pandas as pd

In [ ]:
params = pd.read_csv('./params_tg.csv', sep=';')

In [ ]:
api_id = params['api_id'].values[0]
api_hash = str(params['api_hash'].values[0])

phone = '+' + str(params['phone'].values[0])
username = params['username'].values[0]

client = TelegramClient('session_name', api_id, api_hash)
await client.start()
print("Client Created")
client.run_until_disconnected()

In [ ]:
channels = await client.get_dialogs()
news_channels = ['Коммерсантъ'] # список каналов, из которых мы хотим получать новости
news_channel_ids = []
for channel in channels:
    if channel.title in news_channels:
        news_channel_ids.append(channel.id)

In [ ]:
channel_source = dict(zip(news_channel_ids, news_channels))

In [ ]:
news = []
for channel_id in news_channel_ids:
    channel = PeerChannel(channel_id)
    messages = await client(GetHistoryRequest(
        peer=channel,
        limit=100, # количество сообщений, которые мы хотим получить
        offset_date=None,
        offset_id=0,
        max_id=0,
        min_id=0,
        add_offset=0,
        hash=0))
    for message in messages.messages:
        news.append({
            'date': message.date,
            'text': message.message,
            'source': channel_id
        })

In [ ]:
df = pd.DataFrame(news)

In [ ]:
df['source'] = df['source'].map(channel_source)
df['text'] = df['text'].apply(lambda x: x.replace('\n', ' '))

In [ ]:
client.disconnect()

In [ ]:
df